In [97]:
import cv2
import numpy as np

img=cv2.imread("load1.png")
#print(img.shape, img.size) (1258, 2272) 2858176

ww, hh, rh, r = 640, 400, 0.8, 3
xx1, yy1, xx2, yy2 = int(ww*0.3), int(hh*rh), int(ww*0.7), int(hh*rh)
p1, p2, p3, p4 = [r, hh-r], [ww-r, hh-r], [xx2, yy2], [xx1, yy2]

img = cv2.resize(img, (ww, hh))  # Resize to 640x400
cv2.imshow('oxxostudio', img)
cv2.waitKey(0)

output = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('oxxostudio', output)
cv2.waitKey(0)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
output = cv2.dilate(output, kernel)  # Dilation
cv2.imshow('oxxostudio', output)
cv2.waitKey(0)

output = cv2.GaussianBlur(output, (9, 9), 0)  # Gaussian Blur
#output = cv2.medianBlur(output, 5)  # Median Blur
cv2.imshow('oxxostudio', output)
cv2.waitKey(0)

output = cv2.erode(output, kernel)  # Erosion
cv2.imshow('oxxostudio', output)
cv2.waitKey(0)

output = cv2.Canny(output, 150, 300)  # Canny Edge Detection
cv2.imshow('oxxostudio', output)
cv2.waitKey(0)

   # ROI: Region Of Interest (關注區域)
zero = np.zeros((hh, ww, 1), dtype='uint8')  # ROI
area = [p1, p2, p3, p4]  # bottom left, bottom right, upper right, upper left
pts = np.array(area)
zone = cv2.fillPoly(zero, [pts], 255)
output1 = cv2.bitwise_and(output, zone)  # 使用 bitwise_and
cv2.imshow('oxxostudio', output1)
cv2.waitKey(0)

HOUGH_THRESHOLD, HOUGH_MIN_LINE_LENGTH, HOUGH_MAX_LINE_GAP = 50, 15, 70
lines = cv2.HoughLinesP(output1, 1, np.pi / 180,
                        HOUGH_THRESHOLD, None, HOUGH_MIN_LINE_LENGTH, HOUGH_MAX_LINE_GAP)

done, s1, s2, b1, b2 = 0, 0, 0, 0, 0
img2 = img.copy()
co = (255, 0, 0)

if lines is not None:
    for i in range(0, len(lines)):
        l = lines[i][0]
        x1, y1, x2, y2 = l[0], l[1], l[2], l[3]
        # Calculate the slope and intercept of the line
        s = (y2 - y1) / (x2 - x1)  # s = slope
        b = y1 - s * x1  # y = s * x + b

        cv2.circle(img2, (x1, y1), 3, (0, 255, 0), -1)  # 設定 -1  填滿
        cv2.circle(img2, (x2, y2), 3, (0, 255, 0), -1)  # 設定 -1  填滿
        cv2.line(img2, (x1, y1), (x2, y2), (255, 0, 0), 2)
        print("Q:", s, x1, y1, x2, y2, "done", done)
        cv2.imshow('oxxostudio', img2)
        cv2.waitKey(0)

        if min(x1, x2) < 30 or max(x1, x2) > ww - 30:
            continue

        if done == 0:
            s1 = s
            b1 = b
        else:
            s2 = s
            b2 = b

        done += 1

if done == 3:
    y1, y2 = hh-r, hh-hh*0.175
    x = int(((y1-b1)/s1 + (y1-b2)/s2)/2)
    if abs(x-int(ww/2)-1) > 45:
        co = (0, 255, 255)
    cv2.line(img2, (x, y1), (x, y1-15), co, 2)

    p1 = [(int((y1-b1)/s1), int(y1))]
    p2 = [(int((y2-b1)/s1), int(y2))]
    p3 = [(int((y2-b2)/s2), int(y2))]
    p4 = [(int((y1-b2)/s2), int(y1))]

    zero = np.zeros((hh, ww, 3), dtype='uint8')  # 關注區域
    area = [p1, p2, p3, p4]
    pts = np.array(area)
    mask = cv2.fillPoly(zero, [pts], (0, 50, 0))
    img2 = cv2.addWeighted(img2, 1.0, mask, 1.0, 0)

    img2 = cv2.polylines(img2, [pts], True, (0, 255, 255), 2)  # 繪製多邊形
    cv2.imshow('oxxostudio', img2)
    cv2.waitKey(0)

x, y = int(ww/2)-1, hh-r
cv2.line(img2, (x, y-12), (x, y-2), (255, 0, 0), 2)

for i in range(1, 10):
    x, y = int(ww/2)-1, hh-r
    cv2.line(img2, (x-i*15, y-3), (x-i*15, y-3), (0, 255, 0), 2)
    cv2.line(img2, (x+i*15, y-3), (x+i*15, y-3), (0, 255, 0), 2)
       

cv2.imshow('oxxostudio', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()



Q: 1.2333333333333334 346 320 406 394 done 0
Q: 1.4761904761904763 342 320 384 382 done 1
Q: -0.5454545454545454 105 397 149 373 done 2


In [101]:
import cv2
import numpy as np

def process_frame(frame):
    # 調整為所需尺寸
    ww, hh, rh, r = 640, 400, 0.9, 3
    xx1, yy1, xx2, yy2 = int(ww*0.3), int(hh*rh), int(ww*0.7), int(hh*rh)
    p1, p2, p3, p4 = [r, hh-r], [ww-r, hh-r], [xx2, yy2], [xx1, yy2]
    img = cv2.resize(frame, (ww, hh))
    
    # 轉換為灰階
    output = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 執行影像擴張
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    output = cv2.dilate(output, kernel)
    
    # 套用高斯模糊降低雜訊
    output = cv2.GaussianBlur(output, (7, 7), 0)
    
    # 執行影像侵蝕
    output = cv2.erode(output, kernel)
    
    # 套用Canny邊緣偵測
    output = cv2.Canny(output, 100, 200)
    
    # 創建ROI遮罩(關注區域)
    zero = np.zeros((hh, ww, 1), dtype='uint8')
    area = [p1, p2, p3, p4]  # 左下、右下、右上、左上
    pts = np.array(area)
    zone = cv2.fillPoly(zero, [pts], 255)
    output1 = cv2.bitwise_and(output, zone)
    
    # 套用Hough變換偵測線條
    HOUGH_THRESHOLD, HOUGH_MIN_LINE_LENGTH, HOUGH_MAX_LINE_GAP = 40, 15, 70
    lines = cv2.HoughLinesP(output1, 1, np.pi / 180, HOUGH_THRESHOLD, None, HOUGH_MIN_LINE_LENGTH, HOUGH_MAX_LINE_GAP)
    
    # 處理偵測到的線條
    img2 = img.copy()
    done, s1, s2, b1, b2 = 0, 0, 0, 0, 0
    co = (255, 0, 0)
    lane_points = []
    
    if lines is not None:
        for i in range(0, len(lines)):
            l = lines[i][0]
            x1, y1, x2, y2 = l[0], l[1], l[2], l[3]
            
            # 跳過太靠近邊緣的線條
            if min(x1, x2) < 30 or max(x1, x2) > ww - 30:
                continue
                
            # 計算斜率和截距
            try:
                s = (y2 - y1) / (x2 - x1)  # s = 斜率
                b = y1 - s * x1  # y = s * x + b
                
                if done == 0:
                    s1 = s
                    b1 = b
                    cv2.line(img2, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    lane_points.append(((x1, y1), (x2, y2)))
                elif done == 1:
                    s2 = s
                    b2 = b
                    cv2.line(img2, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    lane_points.append(((x1, y1), (x2, y2)))
                
                done += 1
                if done >= 2:
                    break
            except:
                continue
    
    # 創建車道多邊形並覆蓋
    if done >= 2:
        try:
            y1, y2 = hh-r, hh-hh*0.175
            p1 = [int((y1-b1)/s1), int(y1)]
            p2 = [int((y2-b1)/s1), int(y2)]
            p3 = [int((y2-b2)/s2), int(y2)]
            p4 = [int((y1-b2)/s2), int(y1)]

            # 創建並應用綠色遮罩
            zero = np.zeros((hh, ww, 3), dtype='uint8')
            area = np.array([p1, p2, p3, p4], np.int32).reshape((-1, 1, 2))
            mask = cv2.fillPoly(zero, [area], (0, 50, 0))
            img2 = cv2.addWeighted(img2, 1.0, mask, 0.7, 0)
            
            # 繪製車道多邊形輪廓
            cv2.polylines(img2, [area], True, (0, 255, 255), 2)
            
            # 繪製中心線
            x, y = int(ww/2)-1, hh-r
            cv2.line(img2, (x, y-12), (x, y-2), (255, 0, 0), 2)
            
            # 繪製車道標記
            for i in range(1, 10):
                cv2.line(img2, (x-i*15, y-3), (x-i*15, y-3), (0, 255, 0), 2)
                cv2.line(img2, (x+i*15, y-3), (x+i*15, y-3), (0, 255, 0), 2)
        except:
            pass
    
    return img2

def main():
    # 開啟影片擷取
    cap = cv2.VideoCapture('LaneVideo.mp4')  
    
    
    if not cap.isOpened():
        print("錯誤：無法開啟影片。")
        return
    
    # 獲取影片屬性
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 定義編解碼器並創建VideoWriter物件
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('hw4.mp4', fourcc, fps, (640, 400))
    
    while True:
        ret, frame = cap.read()
        
        # 如果影片結束或出錯則跳出循環
        if not ret:
            break
        
        # 處理影格以進行車道偵測
        processed_frame = process_frame(frame)
        
        # 將處理後的影格寫入輸出影片
        out.write(processed_frame)
        
        # 顯示處理後的影格
        cv2.imshow('車道偵測', processed_frame)
        
        # 如果按下'q'鍵則結束
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

C:\Users\User\AppData\Local\Temp\ipykernel_13708\818387739.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  p1 = [int((y1-b1)/s1), int(y1)]
C:\Users\User\AppData\Local\Temp\ipykernel_13708\818387739.py:81: RuntimeWarning: divide by zero encountered in scalar divide
  p3 = [int((y2-b2)/s2), int(y2)]
C:\Users\User\AppData\Local\Temp\ipykernel_13708\818387739.py:79: RuntimeWarning: invalid value encountered in scalar divide
  p1 = [int((y1-b1)/s1), int(y1)]
